In [1]:
# import all modules

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [15]:
# Device config

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper Parameters

num_epochs = 40
batch_size = 64
learning_rate = 0.001


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Define the VGG16 architecture
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        
        # Convolutional layers
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  # Conv1_1
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),  # Conv1_2
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Pool1
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # Conv2_1
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),  # Conv2_2
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Pool2
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),  # Conv3_1
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),  # Conv3_2
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Pool3
            
            nn.Conv2d(256, 512, kernel_size=3, padding=1),  # Conv4_1
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),  # Conv4_2
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),  # Conv4_3
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Pool4
            
            nn.Conv2d(512, 512, kernel_size=3, padding=1),  # Conv5_1
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),  # Conv5_2
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),  # Conv5_3
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Pool5
        )
        
        # Fully connected layers
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            
            nn.Linear(4096, 1000)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.classifier(x)
        return x

# Define your transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match the input size expected by VGG16
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load your dataset
full_dataset = datasets.ImageFolder(root="C:\\Users\\44778\\OneDrive\\Desktop\\UWE_Docs\\7. Dissertation\\CNN Data Files\\Spectograms", transform=transform)

# Split dataset into training and test sets
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Create DataLoader for training and test sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VGG16().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training the model
num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 200 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

print('Finished Training')

# Evaluation
model.eval()
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for _ in range(len(full_dataset.classes))]
    n_class_samples = [0 for _ in range(len(full_dataset.classes))]

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        
        # Get predictions
        _, predicted = torch.max(outputs, 1)
        
        # Update sample count
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        # Update class-specific counts
        for i in range(labels.size(0)):
            label = labels[i].item()
            pred = predicted[i].item()
            if label == pred:
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    # Calculate and print overall accuracy
    overall_acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {overall_acc:.2f}%')
    
    # Calculate and print class-specific accuracies
    for i in range(len(full_dataset.classes)):
        if n_class_samples[i] > 0:
            class_acc = 100.0 * n_class_correct[i] / n_class_samples[i]
            print(f'Accuracy of {full_dataset.classes[i]}: {class_acc:.2f}%')
        else:
            print(f'No samples for class {full_dataset.classes[i]} in the test set.')


Finished Training
Accuracy of the network: 18.18%
Accuracy of bellbird: 0.00%
Accuracy of haast kiwi: 0.00%
Accuracy of kaka: 0.00%
Accuracy of kea: 0.00%
Accuracy of morepork: 0.00%
Accuracy of other kiwi birds: 100.00%
Accuracy of paradise shelduck: 0.00%
Accuracy of swamphen: 0.00%
